In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
tf.__version__

'1.4.0'

In [3]:
import os
from six.moves.urllib.request import urlopen
import numpy as np

### Data set download

In [4]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [5]:
def main():
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

In [6]:
main()

### Load downloaded CSV files as TF data sets

In [7]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [8]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

### Build a DNN classifer with some hidden units...

In [58]:
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,10,5],
                                          n_classes=3,optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.01,
                                                                                   l1_regularization_strength=0.0001),
                                       activation_fn=tf.nn.sigmoid)
                                          #model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Temp\\tmpm2ljsmq0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000288ACCC0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Define input function

In [59]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

### Train model

In [60]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Temp\tmpm2ljsmq0\model.ckpt.
INFO:tensorflow:loss = 144.229, step = 1
INFO:tensorflow:global_step/sec: 909.039
INFO:tensorflow:loss = 135.805, step = 101 (0.111 sec)
INFO:tensorflow:global_step/sec: 1123.53
INFO:tensorflow:loss = 129.655, step = 201 (0.090 sec)
INFO:tensorflow:global_step/sec: 1052.57
INFO:tensorflow:loss = 117.428, step = 301 (0.095 sec)
INFO:tensorflow:global_step/sec: 1086.89
INFO:tensorflow:loss = 109.521, step = 401 (0.091 sec)
INFO:tensorflow:global_step/sec: 1111.05
INFO:tensorflow:loss = 97.4582, step = 501 (0.091 sec)
INFO:tensorflow:global_step/sec: 1086.89
INFO:tensorflow:loss = 86.9596, step = 601 (0.092 sec)
INFO:tensorflow:global_step/sec: 1041.61
INFO:tensorflow:loss = 73.955, step = 701 (0.095 sec)
INFO:tensorflow:global_step/sec: 1075.21
INFO:tensorflow:loss = 70.3577, step = 801 (0.093 sec)
INFO:tensorflow:global_step/sec: 1086.89
INFO:tensorflow:loss = 73.890

### Define test input function and evaluate accuracy

In [61]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

In [62]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-11-08-00:53:29
INFO:tensorflow:Restoring parameters from C:\Temp\tmpm2ljsmq0\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-08-00:53:29
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.533333, average_loss = 0.634277, global_step = 1000, loss = 19.0283

Test Accuracy: 0.533333


###  Classify two new flower samples

In [ ]:
new_samples = np.array(
      [[6.4, 5.2, 4.5, 1.5],
       [5.8, 1.4, 5.0, 1.7]], dtype=np.float32)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]
predicted_classes = [int(str(x)[-3]) for x in predicted_classes]

print("New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))